In [1]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [2]:
# Load all pickel files
model = load_model('model.h5')

# geo_encoder
with open('ohe.pkl', 'rb') as file:
    ohe = pickle.load(file)

# gender encoder
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender= pickle.load(file)

# scaler encoder
with open('scaler.pkl', 'rb') as file:
    scaler= pickle.load(file)

In [3]:
# Example
input_data = {
    'CreditScore': [600],
    'Geography': ['France'],    
    'Gender': ['Male'],
    'Age': [40],
    'Tenure': [3],
    'Balance': [60000],
    'NumOfProducts': [2],    
    'HasCrCard': [1], 
    'IsActiveMember': [1],    
    'EstimatedSalary': [50000]
}

In [4]:
input_data_df = pd.DataFrame(input_data)

In [5]:
# Converting the input_data labels for geography
geo_ohe = ohe.transform(input_data_df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_ohe, columns = ohe.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
# Converting the input_data labels for gender
input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])

In [7]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
# concate and dropping the geography column
input_data_df = pd.concat([input_data_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [9]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
# Scaling the features using the scaler
input_scaled_data = scaler.transform(input_data_df)
input_scaled_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [24]:
# Make prediction
prediction = model.predict(input_scaled_data)

prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step


array([[0.04806362]], dtype=float32)

In [26]:
# Print the prediction
print(f"Prediction: {prediction}")

Prediction: [[0.04806362]]


In [30]:
prediction_probability = prediction[0][0]
prediction_probability

0.048063617

In [32]:
if prediction_probability >0.5:
    print("The customer is likey to churn")
else:
    print("The customer is not likey to churn")
    

The customer is not likey to churn
